In [ ]:
# For development purposes, reload imported modules when source changes
%load_ext autoreload
%autoreload 2

from spikeforest import spikeextractors as si
from spikeforest import spikewidgets as sw
import os, sys
import numpy as np

from mountainlab_pytools import mlproc as mlp
from mountainlab_pytools import mdaio

from gen_synth_datasets import gen_synth_datasets

import h5py
import json
from matplotlib import pyplot as plt

In [ ]:
#templates='kbucket://b5ecdf1474c5/MEArec/templates/templates_30_Neuronexus-32.h5'
#templates='kbucket://b5ecdf1474c5/MEArec/templates/templates_30_Neuronexus-32.h5'
#templates='kbucket://b5ecdf1474c5/MEArec/templates/templates_30_tetrode_mea.h5'
templates='/mnt/home/jjun/ceph/groundtruth/mearec_synth/neuronexus/templates_30_Neuronexus-32.h5'
    
num_datasets=10
v_K = [10, 20, 40] # list (vector: "v_") of K
v_noise = [10, 20]
v_nChans = [32]
dataset_path = '/mnt/home/jjun/ceph/groundtruth/mearec_synth/neuronexus'

#output 
v_outputdir = [] # stores name of recordingset containing multiple recordings 
v_datasets = []

ds0=dict(
    duration=600,
    templates=templates,
    f_exc=2, f_inh=7,
    min_rate=0.5,
    st_exc=1, st_inh=3,
    channel_ids=None
)
# datasets_noise10_K20_C8
for noise in v_noise:
    for K in v_K:
        for nChans in v_nChans:
            outputdir1 = '{}/datasets_noise{}_K{}_C{}'.format(dataset_path, noise, K, nChans)
            v_outputdir.append(outputdir1)
            datasets = []
            for iDataset in range(1,num_datasets+1):
                ds=dict(        
                    name = '{}_synth'.format('{0:03d}'.format(iDataset)),
                    seed = iDataset,
                    n_exc = int(K/2), n_inh=K-int(K/2),
                    noise_level = noise
                )
                for key in ds0:
                    ds[key]=ds0[key]
                datasets.append(ds)
            v_datasets.append(datasets)
    
#print('DATASETS:')
#for ds in datasets:
#    print(ds['name'])

In [ ]:
print(v_outputdir)

In [ ]:
for datasets, outputdir in zip(v_datasets, v_outputdir):
    gen_synth_datasets(datasets,outdir=outputdir,samplerate=30000)

In [ ]:
IX=si.MdaRecordingExtractor(dataset_directory=v_outputdir[0]+'/001_synth')
OX=si.MdaSortingExtractor(firings_file=v_outputdir[0]+'/001_synth/firings_true.mda')

In [ ]:
sw.TimeseriesWidget(recording=IX).display()

In [ ]:
sw.UnitWaveformsWidget(recording=IX, sorting=OX, snippet_len=100,height=10).plot()